<a href="https://colab.research.google.com/github/hoangcuongnguyen2001/Stocks-Analysis/blob/main/Beta_Calculation_MultipleStocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
import yfinance as yf
import numpy as np


In [25]:
symbols = ['TSLA', 'AAPL', 'SPY']

In [26]:
data = yf.download(symbols, '2017-9-5')['Adj Close']

[*********************100%***********************]  3 of 3 completed


In [27]:
print(data)

                  AAPL         SPY        TSLA
Date                                          
2017-09-05   38.456131  225.606979   23.306000
2017-09-06   38.415798  226.377197   22.968666
2017-09-07   38.261566  226.349670   23.374001
2017-09-08   37.637554  226.083740   22.893333
2017-09-11   38.318516  228.495178   24.246000
...                ...         ...         ...
2022-08-29  161.380005  402.630005  284.820007
2022-08-30  158.910004  398.209991  277.700012
2022-08-31  157.220001  395.179993  275.609985
2022-09-01  157.960007  396.420013  277.160004
2022-09-02  155.809998  392.239990  270.209991

[1259 rows x 3 columns]


In [28]:
price_change = data.pct_change()
price_change['TSLA_net_position'] = price_change.TSLA.cumsum().shift().fillna(0)
price_change['AAPL_net_position'] = price_change.AAPL.cumsum().shift().fillna(0)
price_change['SPY_net_position'] = price_change.SPY.cumsum().shift().fillna(0)
print(price_change)

                AAPL       SPY      TSLA  TSLA_net_position  \
Date                                                          
2017-09-05       NaN       NaN       NaN           0.000000   
2017-09-06 -0.001049  0.003414 -0.014474           0.000000   
2017-09-07 -0.004015 -0.000122  0.017647          -0.014474   
2017-09-08 -0.016309 -0.001175 -0.020564           0.003173   
2017-09-11  0.018093  0.010666  0.059086          -0.017391   
...              ...       ...       ...                ...   
2022-08-29 -0.013690 -0.006612 -0.011351           3.530163   
2022-08-30 -0.015305 -0.010978 -0.024998           3.518812   
2022-08-31 -0.010635 -0.007609 -0.007526           3.493814   
2022-09-01  0.004707  0.003138  0.005624           3.486288   
2022-09-02 -0.013611 -0.010544 -0.025076           3.491912   

            AAPL_net_position  SPY_net_position  
Date                                             
2017-09-05           0.000000          0.000000  
2017-09-06           0.000000 

In [29]:
df = price_change.drop(price_change.index[0])

In [11]:
x = np.array(df['TSLA']).reshape((-1,1))
y = np.array(df['SPY'])

In [31]:
def cm_to_inch(value):
    return value/2.54


import matplotlib.pyplot as plt
import matplotlib.dates as mdates



plt.figure(figsize=(cm_to_inch(50), cm_to_inch(20)))
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(x=df.index,y=df['TSLA_net_position'], mode='lines', name='Tesla'))
fig.add_trace(go.Scatter(x=df.index, y=df['SPY_net_position'],mode='lines',
                    name='S&P 500'))
fig.add_trace(go.Scatter(x=df.index, y=df['AAPL_net_position'],mode='lines',
                    name='Apple'))
fig.update_layout(title='Comparison of price return between S&P 500, Apple and Tesla in 5 years',
                   xaxis_title='Month',
                   yaxis_title='Price index (2017 = 0)')
fig.show()

<Figure size 1417.32x566.929 with 0 Axes>

In [35]:
def covariance(x, y):
    # Finding the mean of the series x and y
    mean_x = sum(x)/float(len(x))
    mean_y = sum(y)/float(len(y))
    # Subtracting mean from the individual elements
    sub_x = [i - mean_x for i in x]
    sub_y = [i - mean_y for i in y]
    numerator = sum([sub_x[i]*sub_y[i] for i in range(len(sub_x))])
    denominator = len(x)-1
    cov = numerator/denominator
    return cov


covar_TSLA = covariance(df.TSLA, df.SPY)
covar_AAPL = covariance(df.AAPL, df.SPY)
variance = np.var(df.SPY)
beta_TSLA = round(covar_TSLA/variance,4)
beta_AAPL = round(covar_AAPL/variance,4)

In [36]:
print('Beta for Tesla: ', beta_TSLA)
print('Beta for Apple: ', beta_AAPL)

Beta for Tesla:  1.4748
Beta for Apple:  1.2369
